# ejercicio CIFAR10 con tensorflow 1.4 o superior

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

[(x_train, y_train), (x_test, y_test)]=tf.keras.datasets.cifar10.load_data ()


In [2]:
num_classes=10
y_train = tf.keras.utils.to_categorical(y_train, num_classes).astype(np.float32)
y_test = tf.keras.utils.to_categorical(y_test, num_classes).astype(np.float32)

In [20]:
print(x_train.shape)
x_train=x_train/255
x_test=x_test/255
#pd.get_dummies(y_train)
print(y_train.shape)
num_labels=10

(50000, 32, 32, 3)
(50000, 10)


In [21]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [23]:
batch_size = 16
image_size= 32
patch_size=5
depth = 32
depth2=64

num_hidden = 4096
num_hidden_2=1024
num_channels=3


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
 # tf_valid_dataset = tf.constant(valid_dataset)
#  tf_test_dataset = tf.constant(x_test)
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  
  # Variables.
#5*5*3*32
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
#5*5*32*64
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))

# fully connected layers
#4096, 1024
  layer3_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden_2], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
#1024,10
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

#keep prob
  keep_prob=tf.placeholder("float")


 
  # Model.
  def model(data,keeps_prob):
        #conv 1
        #32*32*3*32
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    #pooling
    #16*16*32*32
    pool_1=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='VALID')
    
    # Dropout
    #pool_1_drop=tf.nn.dropout(pool_1,keep_prob)
    
    #conv 2
    #16*16*32*64
    conv = tf.nn.conv2d(pool_1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    #pooling
    #8*8*64*64
    pool_2=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='VALID')
    # Dropout
    #pool_2_drop=tf.nn.dropout(pool_2,keep_prob)
    
    #conv 3
    #conv = tf.nn.conv2d(pool_2, layer3_weights, strides=[1, 1, 1, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer3_biases)
    
    #pooling

    
    # Fully connected
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #hidden = tf.nn.relu(tf.matmul(hidden, layer5_weights) + layer5_biases)
        
    hidden=tf.nn.dropout(hidden,keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases


## same model without dropout for testing
  # Model.
  def model_test(data):
        #conv 1
        #32*32*3*32
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    #pooling
    #16*16*32*32
    pool_1=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='VALID')
    
    # Dropout
    #pool_1_drop=tf.nn.dropout(pool_1,keep_prob)
    
    #conv 2
    #16*16*32*64
    conv = tf.nn.conv2d(pool_1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    #pooling
    #8*8*64*64
    pool_2=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='VALID')
    # Dropout
    #pool_2_drop=tf.nn.dropout(pool_2,keep_prob)
    
    #conv 3
    #conv = tf.nn.conv2d(pool_2, layer3_weights, strides=[1, 1, 1, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer3_biases)
    
    #pooling

    
    # Fully connected
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #hidden = tf.nn.relu(tf.matmul(hidden, layer5_weights) + layer5_biases)
        
    return tf.matmul(hidden, layer4_weights) + layer4_biases


   # Training computation.
  logits = model(tf_train_dataset,keep_prob)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  # regulizer
  #regularizers=tf.nn.l2_loss(layer1_weights)+tf.nn.l2_loss(layer2_weights)+tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)+tf.nn.l2_loss(layer5_weights)
  #loss=tf.reduce_mean(loss+beta*regularizers)
  loss=tf.reduce_mean(loss)
  # Optimizer.
  # agregando un learning rate descendiente
  #global_step=tf.Variable(0)
  #start_learning_rate=0.5
  #learning_rate=tf.train.exponential_decay(start_learning_rate,global_step,100000,0.96,staircase=True)
  learning_rate=1e-4  
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    
    
 
  test_prediction = tf.nn.softmax(model_test(tf_test_dataset))

In [27]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    batch_data = x_train[offset:(offset + batch_size), :, :, :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      #print('Validation accuracy: %.1f%%' % accuracy(
       # valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(x_test, y_test))

Initialized
Minibatch loss at step 0: 40.869518
Minibatch accuracy: 0.0%
Minibatch loss at step 50: 17.223484
Minibatch accuracy: 18.8%
Minibatch loss at step 100: 8.884357
Minibatch accuracy: 18.8%
Minibatch loss at step 150: 5.414640
Minibatch accuracy: 25.0%
Minibatch loss at step 200: 4.109779
Minibatch accuracy: 6.2%
Minibatch loss at step 250: 2.481517
Minibatch accuracy: 6.2%
Minibatch loss at step 300: 1.894425
Minibatch accuracy: 43.8%
Minibatch loss at step 350: 1.805705
Minibatch accuracy: 25.0%
Minibatch loss at step 400: 1.757953
Minibatch accuracy: 43.8%
Minibatch loss at step 450: 2.228845
Minibatch accuracy: 6.2%
Minibatch loss at step 500: 2.195414
Minibatch accuracy: 25.0%
Minibatch loss at step 550: 2.183071
Minibatch accuracy: 25.0%
Minibatch loss at step 600: 2.044859
Minibatch accuracy: 18.8%
Minibatch loss at step 650: 2.059647
Minibatch accuracy: 18.8%
Minibatch loss at step 700: 2.079650
Minibatch accuracy: 37.5%
Minibatch loss at step 750: 2.267914
Minibatch a

c:\users\ledra\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
